In [ ]:
!pip install datasets
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import AutoModel, BertTokenizer
import torch
import numpy as np

In [ ]:
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
def tokenize_and_format(sentences):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

  input_ids = []
  attention_masks = []

  for sentence in sentences:
      encoded_dict = tokenizer.encode_plus(
                          sentence,                      
                          add_special_tokens = True, 
                          max_length = 64,           
                          padding = 'max_length',
                          truncation = True,
                          return_attention_mask = True,   
                          return_tensors = 'pt',     
                    )
      input_ids.append(encoded_dict['input_ids'])

      attention_masks.append(encoded_dict['attention_mask'])
  return input_ids, attention_masks

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [ ]:
def get_test_data(df):
  df = df.reset_index(drop=True)

  texts = df.Hypothesis.values
  labels = df.Type.values

  ### tokenize_and_format() is a helper function provided in helpers.py ###
  input_ids, attention_masks = tokenize_and_format(texts)

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)
  test_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(len(texts))]
  test_text = [texts[i] for i in range(len(texts))]
  return test_set, test_text

In [ ]:
from transformers.models.bert.modeling_bert import BertForSequenceClassification
from transformers import RobertaForSequenceClassification
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
# function to get validation accuracy
# ADDED A PARAMETER FOR THE TEXT WHOSE PERFORMANCE IS BEING TESTED FOR FURTHER ANALYSIS
batch_size = 8
def get_test_performance(df,model_path,model_name):
    # Put the model in evaluation mode
    val_set,val_text = get_test_data(df)
    if model_name == 'BERT':
      model = BertForSequenceClassification.from_pretrained(model_path)
    elif model_name == 'ROBERTA':
      model = RobertaForSequenceClassification.from_pretrained(model_path)
    model.to('cuda')
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0

    num_batches = int(len(val_set)/batch_size) + 1

    total_correct = 0

    #### TO GET AVG PERF OF ONE EPOCH #####
    avg_accuracy_list=[]
    avg_precision_list=[]
    avg_recall_list=[]
    avg_f1_list=[]
    preds = []

    for i in range(num_batches):

      end_index = min(batch_size * (i+1), len(val_set))

      batch = val_set[i*batch_size:end_index]
      
      if len(batch) == 0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])
      
      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)
        
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        # ADDED CODE TO PRINT THE INCORRECTLY PREDICTED SENTENCE
        for j in range(len(pred_flat)):
          if pred_flat[j] != labels_flat[j]:
            #pass
            print(f"Incorrectly predicted text: {val_text[i*batch_size+j]}; pred:{pred_flat[j]} ; GT:{labels_flat[j]}")
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        preds.extend(pred_flat)

        ################## classification report for multiclass  ##################
        report = classification_report(labels_flat, pred_flat, output_dict=True)
        #print("Report", report)

        #extract the average values for each metric from the dictionary
        accuracy= report['accuracy']
        precision = report['macro avg']['precision'] 
        recall = report['macro avg']['recall']  
        f1 = report['macro avg']['f1-score']

        avg_accuracy_list.append(accuracy)
        avg_precision_list.append(precision)
        avg_recall_list.append(recall)
        avg_f1_list.append(f1)

        ################## classification report for multiclass  ##################      
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / len(val_set)

    avg_accuracy= sum(avg_accuracy_list) / len(avg_accuracy_list)
    avg_precision= sum(avg_precision_list) / len(avg_precision_list)
    avg_recall= sum(avg_recall_list) / len(avg_recall_list)
    avg_f1=  sum(avg_f1_list) / len(avg_f1_list)

    return avg_val_accuracy, avg_accuracy, avg_precision, avg_recall, avg_f1, preds



#Ensemble model

In [ ]:
test_df = pd.read_csv('../Datasets/Classification/test-set-predictions.csv',sep = ',')
test_df['Type'].replace(['Metaphor', 'Idiom','Simile','Sarcasm'],[0,1,2,3], inplace=True)


In [ ]:
avg_val_acc, avg_acc, avg_precision, avg_recall, avg_f1, preds = get_test_performance(test_df,'../Models/bert-CF-0.06','BERT')
test_df['BERT'] = preds

Incorrectly predicted text: Oh , and there 's one more thing you can be sure of , ’ he added sourly .; pred:1 ; GT:0
Incorrectly predicted text: The tears cut both ways.; pred:0 ; GT:1
Incorrectly predicted text: Armies are the main conventional weapon and have been around for over five millennia in various forms .; pred:0 ; GT:1
Incorrectly predicted text: FROM 1984–90 guesses that cash flow could be higher if assets changed hands or if a firm 's financial structure was changed were made in $ 1.7 trillion - worth of cases in America .; pred:0 ; GT:1
Incorrectly predicted text: I 've been too close up against it .; pred:0 ; GT:1
Incorrectly predicted text: That movie was as disturbing as puppies; pred:0 ; GT:2
Incorrectly predicted text: The judge is as honest as Pinocchio; pred:0 ; GT:2
Incorrectly predicted text: Seeing the old juke box was like visiting a nasty, useless junkyard; pred:2 ; GT:0
Incorrectly predicted text: The river is flowing like a rocket.; pred:2 ; GT:0
Incorrectly

In [ ]:
test_df

,Unnamed: 0,Hypothesis,Type,GPT-3,BERT
0,748,The move was part of a radical ministerial res...,1,Idiom,1
1,572,The clouds were as fluffy as cotton candy.,2,Simile,2
2,668,I wandered lonely as a cloud that floats on hi...,2,Simile,2
3,110,Honesty is a stain,0,Metaphor,0
4,479,I am absolutely in love with spiders and the t...,3,Sarcasm,3
...,...,...,...,...,...
826,423,She was waiting for me to talk about my past a...,1,Idiom,1
827,346,They'd make money hand over fist.,1,Idiom,1
828,113,The dancer floated across the stage.,0,Metaphor,0
829,435,"Could not breed ill-will between them ,",0,Metaphor,0


In [ ]:
print(avg_val_acc, avg_acc, avg_precision, avg_recall, avg_f1)

0.937424789410349 0.9375 0.9110576923076924 0.9087606837606838 0.9015191405816402


In [ ]:
avg_val_acc, avg_acc, avg_precision, avg_recall, avg_f1, preds = get_test_performance(test_df,'../Models/roberta-CF-0.02','ROBERTA')
test_df['ROBERTA'] = preds

Incorrectly predicted text: I wandered lonely as a cloud that floats on high o'er vales and hills.; pred:3 ; GT:2
Incorrectly predicted text: They 'll make a point of finding suffering people.; pred:0 ; GT:1
Incorrectly predicted text: Oh , and there 's one more thing you can be sure of , ’ he added sourly .; pred:1 ; GT:0
Incorrectly predicted text: Armies are the main conventional weapon and have been around for over five millennia in various forms .; pred:0 ; GT:1
Incorrectly predicted text: The ex-slave tasted freedom shortly before she died.; pred:1 ; GT:0
Incorrectly predicted text: FROM 1984–90 guesses that cash flow could be higher if assets changed hands or if a firm 's financial structure was changed were made in $ 1.7 trillion - worth of cases in America .; pred:0 ; GT:1
Incorrectly predicted text: That movie was as disturbing as puppies; pred:0 ; GT:2
Incorrectly predicted text: I asked for your help and you disappeared from radar.; pred:1 ; GT:0
Incorrectly predicted text:

In [ ]:
print(avg_val_acc, avg_acc, avg_precision, avg_recall, avg_f1)

0.8688327316486161 0.8688186813186813 0.8331997863247865 0.8322783119658118 0.8154081450956452


In [ ]:
test_df['Type'].replace([0,1,2,3],['Metaphor', 'Idiom','Simile','Sarcasm'], inplace=True)
test_df['BERT'].replace([0,1,2,3],['Metaphor', 'Idiom','Simile','Sarcasm'], inplace=True)
test_df['ROBERTA'].replace([0,1,2,3],['Metaphor', 'Idiom','Simile','Sarcasm'], inplace=True)

In [ ]:
test_df

,Unnamed: 0,Hypothesis,Type,GPT-3,BERT,ROBERTA
0,748,The move was part of a radical ministerial res...,Idiom,Idiom,Idiom,Idiom
1,572,The clouds were as fluffy as cotton candy.,Simile,Simile,Simile,Simile
2,668,I wandered lonely as a cloud that floats on hi...,Simile,Simile,Simile,Sarcasm
3,110,Honesty is a stain,Metaphor,Metaphor,Metaphor,Metaphor
4,479,I am absolutely in love with spiders and the t...,Sarcasm,Sarcasm,Sarcasm,Sarcasm
...,...,...,...,...,...,...
826,423,She was waiting for me to talk about my past a...,Idiom,Idiom,Idiom,Sarcasm
827,346,They'd make money hand over fist.,Idiom,Idiom,Idiom,Idiom
828,113,The dancer floated across the stage.,Metaphor,Metaphor,Metaphor,Metaphor
829,435,"Could not breed ill-will between them ,",Metaphor,Metaphor,Metaphor,Metaphor


In [ ]:
test_df["Max_pred"] = [None]*test_df.shape[0]

In [ ]:
#majority_voting
from collections import Counter
for i in range(test_df.shape[0]):
  preds = test_df.loc[i,["GPT-3","ROBERTA","BERT"]]
  mapping = dict(Counter(preds))
  max_label = sorted(mapping,key = mapping.get, reverse = True)[0]
  test_df.loc[i,"Max_pred"] = max_label

print("Ensemble accuracy:",1-test_df[test_df.apply(lambda x:x.Type!=x.Max_pred, axis=1)].shape[0]/test_df.shape[0])

In [ ]:
errors = {"GPT-3":{"BERT":0,"ROBERTA":0,"Both":0, "Count":0},"BERT":{"GPT-3":0,"ROBERTA":0,"Both":0,"Count":0},"ROBERTA":{"BERT":0,"GPT-3":0,"Both":0,"Count":0}}
for i in range(test_df.shape[0]):
  actual = test_df.iloc[i].Type
  gpt3 = test_df.iloc[i]["GPT-3"]==actual
  bert = test_df.iloc[i]["BERT"]==actual
  roberta = test_df.iloc[i]["ROBERTA"]==actual
  if(not gpt3):
    errors["GPT-3"]["Count"]+=1
    if(bert and roberta):
      errors["GPT-3"]["Both"]+=1
    else:
      if bert:
        errors["GPT-3"]["BERT"]+=1
      elif roberta:
        errors["GPT-3"]["ROBERTA"]+=1
  if(not bert):
    errors["BERT"]["Count"]+=1
    if(gpt3 and roberta):
      errors["BERT"]["Both"]+=1
    else:
      if gpt3:
        errors["BERT"]["GPT-3"]+=1
      elif roberta:
        errors["BERT"]["ROBERTA"]+=1
  if(not roberta):
    errors["ROBERTA"]["Count"]+=1
    if(bert and gpt3):
      errors["ROBERTA"]["Both"]+=1
    else:
      if bert:
        errors["ROBERTA"]["BERT"]+=1
      elif gpt3:
        errors["ROBERTA"]["GPT-3"]+=1

In [ ]:
errors

{'GPT-3': {'BERT': 7, 'ROBERTA': 4, 'Both': 10, 'Count': 40},
 'BERT': {'GPT-3': 13, 'ROBERTA': 4, 'Both': 16, 'Count': 52},
 'ROBERTA': {'BERT': 7, 'GPT-3': 13, 'Both': 70, 'Count': 109}}